In [ ]:
import MDAnalysis as mda
from MDAnalysis.analysis.dihedrals import Dihedral, Ramachandran, Janin
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def read_gyration_df(directory):
    gyration_df = pd.read_csv(f'{directory}/gyration.xvg', sep = '\\s+', header=None, skiprows=[i for i in range(0, 27)])
    gyration_df.columns = ['time', 'gyr', 'g', 'h', 'j']
    gyration_df.drop(['g', 'h', 'j'], axis=1, inplace = True)
    gyration_df['time'] = gyration_df['time']/1000

    return gyration_df


def read_mdmat(directory):
    mat_df = pd.read_csv(f'{directory}/mat.dat', header=None, sep = '\s+')
    mat_df.columns = ['ai', 'aj', 'distance', 'probability']

    return mat_df


def read_rama(directory):
    rama = pd.read_csv(f'{directory}/rama.xvg', sep = '\\s+', header=None, skiprows=[i for i in range(0, 34)])
    rama.columns = ['phi', 'psi', 'residue']

    return rama

In [ ]:
#greta_rama_amino_list = rama_greta_harp0['residue'].unique()


In [ ]:
def make_analysis(**parameters):
    analysis_dict = {}
    for name, para in parameters.items():
        analysis_dict[name] = {}
        %cd {para['directory']}
        
        if para['make_pbc'] == True:
            gmx_pbc_string = f'echo -e "0\n" | gmx_mpi trjconv -f {para["trajectory"]} -s {para["tpr"]} -pbc mol -o nopbc.xtc'
            !{gmx_pbc_string}

        if para['make_gyration'] == True:
            gmx_gyrate_string = f'echo -e "4\n" | gmx_mpi gyrate -f nopbc.xtc -s {para["tpr"]} -o gyration.xvg'
            !{gmx_gyrate_string}

        if para['make_contacts'] == True:
            gmx_mdmat_string = f'echo -e "2\n" | gmx_mpi mdmat -f nopbc.xtc -s {para["tpr"]}'
            !{gmx_mdmat_string}
        
        if para['make_ramachandran'] == True:
            gmx_rama_string = f'gmx_mpi rama -f nopbc.xtc -s {para["tpr"]} -o rama.xvg'
            !{gmx_rama_string}

        analysis_dict[name]['gyration'] = read_gyration_df(para['directory'])
        analysis_dict[name]['mdmat'] = read_mdmat(para['directory'])
        analysis_dict[name]['ramachandran'] = read_rama(para['directory'])
    
    return analysis_dict

In [ ]:
def plotly_gyration(monomer_analysis):
    group_data, group_labels = [], []
    for name, data in monomer_analysis.items():
        group_data.append(data['gyration']['gyr'])
        group_labels.append(name)

    return group_data, group_labels


def prepare_plotly_heatmap(df):
    return {'z': df.probability.tolist(),
            'x': df.ai.tolist(),
            'y': df.aj.tolist()}


def plotly_heatmap(monomer_analysis):
    plainMD_mat = monomer_analysis['plainMD']['mdmat']
    mdmat_names = list(monomer_analysis.keys())
    mdmat_names.remove('plainMD')
    mdmat_names.remove('random_coil')
    rows = list(range(len(mdmat_names)))
    rows = [c+1 for c in rows]
    subplot_titles = []
    for name in mdmat_names:
        subplot_titles.append('PlainMD')
        subplot_titles.append(name)
        subplot_titles.append('Diff')
    fig = make_subplots(rows = max(rows), cols = 3, subplot_titles=subplot_titles)
    for mdmat, row in zip(mdmat_names, rows):
        diff_mat = pd.DataFrame()
        diff_mat['ai'] = monomer_analysis[mdmat]['mdmat']['ai']
        diff_mat['aj'] = monomer_analysis[mdmat]['mdmat']['aj']
        diff_mat['probability'] = monomer_analysis[mdmat]['mdmat']['probability'] - plainMD_mat['probability']
        print(f'{mdmat}: ',sum(diff_mat['probability']*diff_mat['probability']))
        fig.add_trace(go.Heatmap(prepare_plotly_heatmap(plainMD_mat)), row=row, col=1)
        fig.add_trace(go.Heatmap(prepare_plotly_heatmap(monomer_analysis[mdmat]['mdmat'])), row=row, col=2)
        fig.add_trace(go.Heatmap(prepare_plotly_heatmap(diff_mat)), row=row, col=3)


    fig.update_layout(height = max(rows)*500, width = 3*500)

    return fig


def matplotlib_rama(monomer_analysis):
    rama_names = list(monomer_analysis.keys())
    cols = len(rama_names)
    residue_list = list(monomer_analysis['plainMD']['ramachandran']['residue'].unique())
    rows = list(range(len(residue_list)))
    fig, axs = plt.subplots(max(rows)+1, cols, figsize = (cols*10, max(rows)*10))

    for name, col in zip(rama_names, list(range(cols))):
        for residue, row in zip(residue_list, rows):
            temp_df = monomer_analysis[name]['ramachandran'].loc[monomer_analysis[name]['ramachandran']['residue'] == residue]
            axs[row, col].scatter(temp_df['phi'], temp_df['psi'], alpha = 0.1)
            axs[row, col].set_xlim(-180, 180)
            axs[row, col].set_ylim(-180, 180)
            axs[row, col].set_title(f'{name} {residue}')
    plt.close()
    return fig


# CONTROLS

## Plain MD

In [ ]:
plain_MD_para = {
    'tpr':'topol_noh.tpr',
    'structure':'reduced-noh.gro',
    'trajectory':'traj_red_noh.xtc',
    'directory':'/home/emanuele/ABeta/markov',
    'make_pbc':False,
    'make_gyration': False,
    'make_contacts': False,
    'make_ramachandran': False
}

## Random Coil

In [ ]:

rc_para = {
    'tpr':'prod_abeta_greta.tpr',
    'structure':'prod_abeta_greta-posre.gro',
    'trajectory':'prod_abeta_greta.xtc',
    'directory':'/home/emanuele/ABeta/random_coil/monomer_test/native_278K',
    'make_pbc': False,
    'make_gyration': False,
    'make_contacts': False,
    'make_ramachandran': False
}


# TESTS

In [ ]:
e300 = {
    'tpr':'prod_abeta_greta.tpr',
    'structure':'prod_abeta_greta-posre.gro',
    'trajectory':'prod_abeta_greta.xtc',
    'directory':'/home/emanuele/ABeta/greta_co5.5_ex2/epsilon_300_merge/monomer_test/native_278K_new',
    'make_pbc': False,
    'make_gyration': False,
    'make_contacts': False,
    'make_ramachandran': False
}

In [ ]:
e320 = {
    'tpr':'prod_abeta_greta.tpr',
    'structure':'prod_abeta_greta-posre.gro',
    'trajectory':'prod_abeta_greta.xtc',
    'directory':'/home/emanuele/ABeta/greta_co5.5_ex2/epsilon_320_merge/monomer_test/native_278K_pe21',
    'make_pbc': True,
    'make_gyration': True,
    'make_contacts': True,
    'make_ramachandran': True
}

In [ ]:
double300_320 = {
    'tpr':'prod_abeta_greta.tpr',
    'structure':'prod_abeta_greta-posre.gro',
    'trajectory':'prod_abeta_greta.xtc',
    'directory':'/home/emanuele/ABeta/greta_co5.5_ex2/double_epsilon_idp300_fib320_merge/monomer_test/native_278K_pe21',
    'make_pbc': True,
    'make_gyration': True,
    'make_contacts': True,
    'make_ramachandran': True
}

In [ ]:
monomer_analysis = make_analysis(plainMD = plain_MD_para, random_coil = rc_para, e300 = e300, e320 = e320, double300_320 = double300_320)

In [ ]:
group_data, group_labels = plotly_gyration(monomer_analysis)

#colors = ['rgb(230, 142, 204)', 'rgb(35, 181, 230)', 'rgb(230, 149, 101)', 'rgb(230, 205, 88)', 'rgb(65, 230, 175)']

fig = ff.create_distplot(group_data, group_labels, bin_size=.01, show_rug=False)#, colors=colors)
fig.update_layout(autosize = False , width = 800, height = 800, plot_bgcolor = 'white', font = dict(size = 30))#, legend  = dict(yanchor = 'top', y = 0.99, xanchor = 'left', x = 0.01))
fig.update_yaxes(title = 'pdf', showline = True, linecolor = 'black', rangemode = 'tozero', showticklabels = False)
fig.update_xaxes(title = 'Radius of gyration (nm)', showline = True, linecolor = 'black')

#fig.write_image("gyr_histograms.png")
fig.show()


In [ ]:
plotly_heatmap(monomer_analysis)

In [ ]:
rama_plot = matplotlib_rama(monomer_analysis)
rama_plot